In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from constants import DATA_END_DATE, DATA_START_DATE
from db_helper_functions import (
    get_stock_news_with_finbert_tone_scores_from_db,
    get_stock_news_with_finbert_whole_article_scores_from_db,
    get_stock_news_with_finbert_scores_from_db,
)
from sklearn.model_selection import TimeSeriesSplit


ticker = "AAPL"


In [3]:
df_opts = [
    get_stock_news_with_finbert_tone_scores_from_db(ticker),
    get_stock_news_with_finbert_whole_article_scores_from_db(ticker),
    get_stock_news_with_finbert_scores_from_db(ticker),
]
df = df_opts[1]
grouped_sentiments = df.groupby("date", as_index=False).agg(
    {"positive": "mean", "negative": "mean", "neutral": "mean"}
)
price_history = (
    yf.Ticker(ticker).history(start=DATA_START_DATE, end=DATA_END_DATE).reset_index()
)
price_history.columns = [x.lower() for x in price_history.columns]
price_history["date"] = price_history["date"].dt.date
price_history.head()
combo_df = pd.merge(
    price_history, grouped_sentiments, left_on="date", right_on="date", how="left"
)
combo_df = combo_df.sort_values(by="date", ascending=True)
combo_df = combo_df.set_index("date")
combo_df

c:\Users\vince\MADS\Milestone 2\w24-milestone2-team18-leevw-steveso-raulmart\venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,open,high,low,close,volume,dividends,stock splits,positive,negative,neutral
date,,,,,,,,,,
2019-01-04,34.636174,35.599556,34.461233,35.530056,234428400,0.0,0.0,0.358300,0.426100,0.215600
2019-01-07,35.635491,35.666646,34.964479,35.450962,219111200,0.0,0.0,0.117100,0.686500,0.196400
2019-01-08,35.841586,36.383190,35.592355,36.126766,164101200,0.0,0.0,0.252700,0.531600,0.215600
2019-01-09,36.256177,37.032634,35.858366,36.740265,180396400,0.0,0.0,0.268725,0.414900,0.316400
2019-01-10,36.546151,36.898432,36.153130,36.857693,143122800,0.0,0.0,0.405500,0.202000,0.392500
...,...,...,...,...,...,...,...,...,...,...
2022-12-27,130.483232,130.513026,127.841385,129.142441,69007800,0.0,0.0,0.236418,0.375418,0.388145
2022-12-28,128.784907,130.135625,125.010850,125.179688,85438400,0.0,0.0,0.180425,0.423800,0.395763
2022-12-29,127.116366,129.589368,126.858146,128.725311,75703700,0.0,0.0,0.292311,0.475433,0.232256


In [12]:
import pandas as pd
import datetime as dt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader


In [11]:
sentiments_df = combo_df[['close', 'positive', 'negative', 'neutral']]

# Aggregate data into months, using the average of columns as the values (ignoring NaN)
def aggregate_monthly_data(df):
    df.index = pd.to_datetime(df.index)
    df['date_monthly'] = df.index.to_period('M').strftime('%Y-%m')
    monthly_data = df.groupby('date_monthly').mean()
    monthly_data = monthly_data.sort_index()
    return monthly_data
monthly_df = aggregate_monthly_data(sentiments_df.copy())
monthly_df

,close,positive,negative,neutral
date_monthly,,,,
2019-01,37.049161,0.291351,0.399724,0.308912
2019-02,41.284196,0.305565,0.350038,0.344378
2019-03,44.114186,0.286235,0.269212,0.444551
2019-04,48.259178,0.339818,0.350140,0.310044
2019-05,46.151073,0.275238,0.367214,0.357551
...,...,...,...,...
2022-09,151.706833,0.221570,0.393089,0.385339
2022-10,143.784985,0.244999,0.371511,0.383489
2022-11,144.813802,0.236016,0.361605,0.402381


In [ ]:
window_size = 3
